# 🔧 Environment Setup

In [1]:
!pip install -U numpy==1.26.4 pandas scikit-learn     langchain langchain-openai langchain-community     qdrant-client neo4j rapidfuzz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 549.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 710.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 416.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 635.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 22.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 25.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 🛠 Fix numpy bool compatibility (temporary workaround)

In [2]:
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool


/tmp/ipykernel_122/2245577656.py:2: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'bool'):


## 📁 Load Telecom Data

In [49]:
import pandas as pd

templates_df = pd.read_csv('templates.csv')
projects_df = pd.read_csv('projects_tasks.csv')
templates_df.head(), projects_df.head()


(    template_id          template_name  default_duration_days  task_count  \
 0  TEMPLATE-001  5G Tower Installation                      2           5   
 1  TEMPLATE-002        Fiber Trenching                     13           3   
 2  TEMPLATE-003   Site Acceptance Test                      2           3   
 3  TEMPLATE-004        Fiber Trenching                     10           3   
 4  TEMPLATE-005        RF Optimization                      8           4   
 
                 required_roles  
 0         Technician, Engineer  
 1    Engineer, Project Manager  
 2                 Engineer, QA  
 3  Technician, Project Manager  
 4          QA, Project Manager  ,
    project_id      task_id   template_id                          task_name  \
 0  PROJ-00001  TASK-000001  TEMPLATE-009        Power Backup Setup - Survey   
 1  PROJ-00001  TASK-000002  TEMPLATE-009    Power Backup Setup - Excavation   
 2  PROJ-00001  TASK-000003  TEMPLATE-009  Power Backup Setup - Cable Laying   
 3  P

## 🧠 Neo4j Graph Ingestion

In [54]:
import pandas as pd
from neo4j import GraphDatabase

# Load data
templates_df = pd.read_csv("templates.csv")
projects_df = pd.read_csv("projects_tasks.csv")

# Connect to Neo4j
uri = "bolt://graph-neo4j:7687"
auth = ("neo4j", "password")
driver = GraphDatabase.driver(uri, auth=auth)
driver.verify_connectivity()
print("✅ Connected to Neo4j")

# Helper: convert pandas row to Cypher properties string
def row_to_cypher_props(row):
    props = []
    date_fields = {"start_date", "end_date", "sla_due"}  # define date fields

    for key, value in row.items():
        if pd.isna(value):
            continue  # skip NaNs

        if key in date_fields:
            try:
                # Check if it looks like a date
                pd.to_datetime(value)
                props.append(f"{key}: date('{value}')")
            except:
                continue  # skip malformed dates

        elif isinstance(value, str):
            value_str = value.replace("'", "\\'")
            props.append(f"{key}: '{value_str}'")
        else:
            props.append(f"{key}: {value}")
    return ", ".join(props)


# Define dynamic merge pattern for templates
def create_template_query(row):
    return f"MERGE (:Template {{ {row_to_cypher_props(row)} }})"

# Define dynamic merge pattern for tasks
def create_task_query(row):
    task_props = row_to_cypher_props(row[['task_id', 'task_name', 'status', 'risk', 'owner', 'location', 'start_date', 'end_date', 'sla_due']])
    project_id = row['project_id']
    template_id = row['template_id']
    dependency = row.get('dependency', None)

    queries = [
        f"MERGE (p:Project {{id: '{project_id}'}})",
        f"MERGE (t:Task {{ {task_props} }})",
        f"MERGE (tpl:Template {{id: '{template_id}'}})",
        f"MERGE (p)-[:HAS_TASK]->(t)",
        f"MERGE (t)-[:USES_TEMPLATE]->(tpl)"
    ]
    if pd.notna(dependency):
        queries.append(f"MERGE (d:Task {{id: '{dependency}'}})")
        queries.append(f"MERGE (t)-[:DEPENDS_ON]->(d)")
    return "\n".join(queries)

with driver.session() as session:
    # ❌ Clear existing database
    session.run("MATCH (n) DETACH DELETE n")
    print("🧹 Cleared existing Neo4j data.")

    # Insert templates
    for i, row in templates_df.iterrows():
        cypher = create_template_query(row)
        session.run(cypher)
        if i % 100 == 0:
            print(f"Inserted {i} templates")

    # Insert projects/tasks
    for i, row in projects_df.iterrows():
        cypher = create_task_query(row)
        session.run(cypher)
        if i % 500 == 0:
            print(f"Inserted {i} tasks")



driver.close()
print("✅ Data ingestion complete!")


✅ Connected to Neo4j
🧹 Cleared existing Neo4j data.
Inserted 0 templates
Inserted 0 tasks
Inserted 500 tasks
Inserted 1000 tasks
Inserted 1500 tasks
✅ Data ingestion complete!


## 📦 Vector Store Ingestion to Qdrant

In [55]:
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

# Monkey patch fix for NumPy
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool

# Embeddings (uses env var OPENAI_API_KEY)
embedding_model = OpenAIEmbeddings()

# Connect to Qdrant container by name (inside Docker)
qdrant_client = QdrantClient(
    host="vector-qdrant", port=6333
)

# Convert your data
texts = [f"{row.template_name} - Roles: {row.required_roles}" for _, row in templates_df.iterrows()]
metadata = templates_df.to_dict(orient='records')

# Correct way to build vector store
vectorstore = Qdrant.from_texts(
    texts=texts,
    embedding=embedding_model,
    metadatas=metadata,
    collection_name="telecom_templates",
    url="http://vector-qdrant:6333"  # 👈 Instead of `qdrant_client=...`
)


## 🧠 Agent with Graph + Vector RAG

In [68]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
from langchain.chains import RetrievalQA
from langchain.graphs import Neo4jGraph

# Initialize LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Neo4j setup
uri = "bolt://graph-neo4j:7687"
graph = Neo4jGraph(url=uri, username="neo4j", password="password")

# Graph chain
cypher_chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)

# Vector retriever chain
retriever_chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=vectorstore.as_retriever(), verbose=True
)

# ✅ Use plain Tool (string input), not StructuredTool
tools = [
    Tool(
        name="Graph QA",
        func=cypher_chain.run,
        description="Use to answer questions about projects, tasks, or relationships. Ask in plain English."
    ),
    Tool(
        name="Template Lookup",
        func=retriever_chain.run,
        description="Use for questions about templates or required roles. Ask in plain English."
    )
]

# Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


## 🔍 Ask Questions to Agent

In [71]:
response = agent.run("Which tasks have the highest SLAs?")
print(response)




> Entering new AgentExecutor chain...
To determine which tasks have the highest SLAs, I need to query the system for information about tasks and their associated SLAs. 
Action: Graph QA
Action Input: "List tasks with their SLAs and identify the ones with the highest SLAs."

> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (t:Task)
RETURN t.task_name, t.sla_due
ORDER BY t.sla_due DESC

Full Context:
[{'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}, {'t.task_name': None, 't.sla_due': None}]

> Finished chain.

Observation: I don't know the answer.
Thought:It seems I need to refine my query to get the information about tasks and their SLAs